In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

def dia():
    from datetime import date, timedelta
    dia = date.today() + timedelta(0)
    return dia

def drop_reset_index(df):
    df = df.dropna()
    df = df.reset_index(drop=True)
    df.index += 1
    return df
    
def grafico(df, nome):
    df = df.reset_index(drop=True)
    df.index += 1
    df['Profit_acu'] = df.Profit.cumsum()
    profit = round(df.Profit_acu.tail(1).item(),2)
    ROI = round((df.Profit_acu.tail(1)/len(df)*100).item(),2)
    df.Profit_acu.plot(title=nome, xlabel='Entradas', ylabel='Stakes')
    print("Profit:",profit,"stakes em", len(df),"jogos")
    print("ROI:",ROI,"%")

## Instalando as Dependências do Ubuntu

In [2]:
# %%shell
# # Ubuntu no longer distributes chromium-browser outside of snap
# #
# # Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# # Add debian buster
# cat > /etc/apt/sources.list.d/debian.list <<'EOF'
# deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
# deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
# deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
# EOF

# # Add keys
# apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
# apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
# apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

# apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
# apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
# apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# # Prefer debian repo for chromium* packages only
# # Note the double-blank lines between entries
# cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
# Package: *
# Pin: release a=eoan
# Pin-Priority: 500

# Package: *
# Pin: origin "deb.debian.org"
# Pin-Priority: 300

# Package: chromium*
# Pin: origin "deb.debian.org"
# Pin-Priority: 700
# EOF

# # Install chromium and chromium-driver
# apt-get update
# apt-get install chromium chromium-driver

# # Install selenium
# pip install selenium

## Configuração do Web-Driver

In [3]:
import time
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.headless = True

from selenium.webdriver.chrome.service import Service
wd_Chrome = webdriver.Chrome("/usr/bin/chromedriver", options=options)

## Iniciando a Raspagem de Dados

In [4]:
# Com o WebDrive a gente consegue a pedir a página (URL)
wd_Chrome.get("https://www.flashscore.com/")

In [5]:
# Fechando Botão de Cookies
try:
    button_cookies = wd_Chrome.find_element(By.CSS_SELECTOR,'button#onetrust-accept-btn-handler')
    button_cookies.click()
except:
    pass

# Selecionando o Dia de Amanhã
time.sleep(3)
wd_Chrome.find_element(By.CSS_SELECTOR,'button.calendar__navigation--tomorrow').click()
time.sleep(3)

# Pegando o ID dos Jogos
id_jogos = []
jogos = wd_Chrome.find_elements(By.CSS_SELECTOR,'div.event__match--scheduled')

for i in jogos:
    id_jogos.append(i.get_attribute("id"))

# Exemplo de ID de um jogo: 'g_1_Gb7buXVt'    
id_jogos = [i[4:] for i in id_jogos]

jogo = {'Date':[],'Time':[],'Country':[],'League':[],'Home':[],'Away':[],'Odd_H':[],'Odd_D':[],'Odd_A':[],'Odd_Over15':[],'Odd_Under15':[],'Odd_Over25':[],'Odd_Under25':[],'Odd_BTTS_Yes':[],'Odd_BTTS_No':[]}

ElementClickInterceptedException: Message: element click intercepted: Element <button class="calendar__navigation calendar__navigation--tomorrow" title="Next day">...</button> is not clickable at point (743, 402). Other element would receive the click: <button id="onetrust-accept-btn-handler">...</button>
  (Session info: headless chrome=114.0.5735.248)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x009AA813+48355]
	(No symbol) [0x0093C4B1]
	(No symbol) [0x00845358]
	(No symbol) [0x00875062]
	(No symbol) [0x00873B98]
	(No symbol) [0x0087239B]
	(No symbol) [0x0087179E]
	(No symbol) [0x00869F5C]
	(No symbol) [0x0088A73C]
	(No symbol) [0x00869A36]
	(No symbol) [0x0088AA94]
	(No symbol) [0x0089C922]
	(No symbol) [0x0088A536]
	(No symbol) [0x008682DC]
	(No symbol) [0x008693DD]
	GetHandleVerifier [0x00C0AABD+2539405]
	GetHandleVerifier [0x00C4A78F+2800735]
	GetHandleVerifier [0x00C4456C+2775612]
	GetHandleVerifier [0x00A351E0+616112]
	(No symbol) [0x00945F8C]
	(No symbol) [0x00942328]
	(No sym

In [ ]:
# Pegando as Informacoes Básicas do Jogo
for link in tqdm(id_jogos, total=len(id_jogos)):
    wd_Chrome.get(f'https://www.flashscore.com/match/{link}/#/match-summary')
    
    try:
        Date = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__startTime').text.split(' ')[0]
        Time = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__startTime').text.split(' ')[1]
        Country = wd_Chrome.find_element(By.CSS_SELECTOR,'span.tournamentHeader__country').text.split(':')[0]
        League = wd_Chrome.find_element(By.CSS_SELECTOR,'span.tournamentHeader__country')
        League = League.find_element(By.CSS_SELECTOR,'a').text
        Home = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__home')
        Home = Home.find_element(By.CSS_SELECTOR,'div.participant__participantName').text
        Away = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__away')
        Away = Away.find_element(By.CSS_SELECTOR,'div.participant__participantName').text
    except:
        pass

# Match Odds
    try:
        wd_Chrome.get(f'https://www.flashscore.com/match/{link}/#/odds-comparison/1x2-odds/full-time')
        time.sleep(2)
        
        linhas = wd_Chrome.find_elements(By.CSS_SELECTOR,'div.ui-table__row')
        
        for linha in linhas:
                Odd_H = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[0].text)
                Odd_D = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[1].text) 
                Odd_A = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[2].text)
    except:
        pass

# Over e Under 2.5
    try:
        wd_Chrome.get(f'https://www.flashscore.com/match/{link}/#/odds-comparison/over-under/full-time')
        time.sleep(2)
        
        linhas_over_under = wd_Chrome.find_elements(By.CSS_SELECTOR,'div.ui-table.oddsCell__odds')
        for i in linhas_over_under:
            linhas = i.find_elements(By.CSS_SELECTOR,'div.ui-table__row')
            for linha in linhas:
                try:
                    qtd_gols = linha.find_element(By.CSS_SELECTOR,'span.oddsCell__noOddsCell').text.replace('.','')
                    if qtd_gols == '05':
                        Odd_Over05 = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[0].text)
                        Odd_Under05 = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[1].text)
                    if qtd_gols == '15':
                        Odd_Over15 = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[0].text)
                        Odd_Under15 = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[1].text)
                        #print(Odd_Under15)
                    if qtd_gols == '25':
                        Odd_Over25 = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[0].text)
                        Odd_Under25 = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[1].text)
                except:
                    pass
    except:
        pass

# Ambas Marcam (BTTS)
    try:
        wd_Chrome.get(f'https://www.flashscore.com/match/{link}/#/odds-comparison/both-teams-to-score/full-time')
        time.sleep(2)
    
        linhas = wd_Chrome.find_elements(By.CSS_SELECTOR,'div.ui-table__row')
        
        for linha in linhas:
                BTTS_Yes = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[0].text)
                BTTS_No = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[1].text)
    except:
        pass    
    
    try:
        print(Date,Time,Country,League,Home,Away,Odd_H,Odd_D,Odd_A,Odd_Over15,Odd_Under15,Odd_Over25,Odd_Under25,BTTS_Yes,BTTS_No) 

        jogo['Date'].append(Date)
        jogo['Time'].append(Time)
        jogo['Country'].append(Country)
        jogo['League'].append(League)
        jogo['Home'].append(Home)
        jogo['Away'].append(Away)
        jogo['Odd_H'].append(Odd_H)
        jogo['Odd_D'].append(Odd_D)
        jogo['Odd_A'].append(Odd_A)
        jogo['Odd_Over15'].append(Odd_Over15)
        jogo['Odd_Under15'].append(Odd_Under15)
        jogo['Odd_Over25'].append(Odd_Over25)
        jogo['Odd_Under25'].append(Odd_Under25)
        jogo['Odd_BTTS_Yes'].append(BTTS_Yes)
        jogo['Odd_BTTS_No'].append(BTTS_No)

    except:
        pass

 39%|███▉      | 61/157 [08:32<12:49,  8.02s/it]

26.07.2023 15:30 SWITZERLAND SUPER LEAGUE - ROUND 1 Lausanne Ouchy Lugano 3.4 3.6 2.0 1.22 4.0 1.75 2.05 1.63 2.2


 39%|███▉      | 62/157 [08:41<12:58,  8.20s/it]

26.07.2023 20:00 USA USL CHAMPIONSHIP Pittsburgh Indy Eleven 1.63 3.75 5.0 1.28 3.5 1.9 1.85 1.9 1.85


 40%|████      | 63/157 [08:50<13:13,  8.44s/it]

...

26.07.2023 21:30 WORLD CLUB FRIENDLY Real Madrid (Esp) Manchester Utd (Eng) 2.3 3.75 2.8 1.18 4.75 1.6 2.3 1.5 2.5


 99%|█████████▉| 156/157 [20:15<00:08,  8.26s/it]

26.07.2023 23:30 WORLD CLUB FRIENDLY Arsenal (Eng) Barcelona (Esp) 2.47 3.62 2.66 1.19 4.0 1.65 2.08 1.57 2.34


100%|██████████| 157/157 [20:22<00:00,  7.79s/it]

26.07.2023 09:00 WORLD CLUB FRIENDLY WOMEN Sparta Prague W (Cze) Ferencvaros W (Hun) 2.47 3.62 2.66 1.19 4.0 1.65 2.08 1.57 2.34


## Gerando DataFrame com os Jogos do Dia

In [ ]:
df = pd.DataFrame(jogo)
df = drop_reset_index(df)
df

,Date,Time,Country,League,Home,Away,Odd_H,Odd_D,Odd_A,Odd_Over15,Odd_Under15,Odd_Over25,Odd_Under25,Odd_BTTS_Yes,Odd_BTTS_No
1,26.07.2023,14:00,EUROPE,CHAMPIONS LEAGUE - QUALIFICATION - QUARTER-FINALS,Aris (Cyp),BATE (Blr),1.57,3.75,6.00,1.38,2.90,2.20,1.63,2.20,1.63
2,26.07.2023,14:00,EUROPE,CHAMPIONS LEAGUE - QUALIFICATION - QUARTER-FINALS,Sheriff Tiraspol (Mda),Maccabi Haifa (Isr),2.95,3.00,2.50,1.45,2.60,2.40,1.53,2.00,1.75
3,26.07.2023,15:15,EUROPE,CHAMPIONS LEAGUE - QUALIFICATION - QUARTER-FINALS,Rakow (Pol),Qarabag (Aze),2.00,3.30,3.75,1.47,2.55,2.45,1.52,2.10,1.68
4,26.07.2023,15:45,EUROPE,CHAMPIONS LEAGUE - QUALIFICATION - QUARTER-FINALS,Klaksvik (Fai),Hacken (Swe),4.75,3.60,1.72,1.36,3.00,2.10,1.68,2.00,1.75
5,26.07.2023,15:45,EUROPE,CHAMPIONS LEAGUE - QUALIFICATION - QUARTER-FINALS,Ludogorets (Bul),O. Ljubljana (Slo),1.53,4.00,6.00,1.30,3.30,1.95,1.80,2.05,1.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,26.07.2023,21:00,WORLD,CLUB FRIENDLY,Crystal Palace (Eng),Millonarios (Col),2.02,3.40,3.34,1.28,3.50,1.87,1.87,1.75,2.03
154,26.07.2023,21:15,WORLD,CLUB FRIENDLY,Chelsea (Eng),Newcastle (Eng),2.45,3.60,2.65,1.22,4.20,1.70,2.15,1.57,2.35
155,26.07.2023,21:30,WORLD,CLUB FRIENDLY,Real Madrid (Esp),Manchester Utd (Eng),2.30,3.75,2.80,1.18,4.75,1.60,2.30,1.50,2.50
156,26.07.2023,23:30,WORLD,CLUB FRIENDLY,Arsenal (Eng),Barcelona (Esp),2.47,3.62,2.66,1.19,4.00,1.65,2.08,1.57,2.34


In [ ]:
df = df.rename(columns={'Odd_H': 'FT_Odd_H',
                        'Odd_D': 'FT_Odd_D',
                        'Odd_A': 'FT_Odd_A',
                        'Odd_Over15': 'FT_Odd_Over15',
                        'Odd_Under15': 'FT_Odd_Under15',
                        'Odd_Over25': 'FT_Odd_Over25',
                        'Odd_Under25': 'FT_Odd_Under25',
                        'Odd_BTTS_Yes': 'FT_Odd_BTTS_Yes',
                        'Odd_BTTS_No': 'FT_Odd_BTTS_No',
                        })

In [ ]:
# Gerando os dados para excel
from datetime import date, datetime, timedelta
data = datetime.today() + timedelta(1)
amanha = data.strftime('%Y-%m-%d')
nome = 'Jogos_do_Dia_FlashScore.xlsx'

df.to_excel(f'G:\\Meu Drive\\Programacao\\futPythonTrader\\Jogos do dia\\{amanha}_{nome}', index = False)